In [1]:
from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
import pandas as pd
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score, classification_report, precision_recall_curve
import numpy as np
import plotly.express as px
# import shap
import seaborn as sns
from IPython import display
import matplotlib.pyplot as plt
import awswrangler as wr
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    classification_report,
    multilabel_confusion_matrix,
)
from catboost.utils import select_threshold
from skmultilearn.model_selection.iterative_stratification import (
    IterativeStratification,
)
from sklearn.model_selection import StratifiedKFold, KFold
from tqdm import tqdm
from catboost.utils import get_roc_curve, select_threshold
from torchmetrics.classification import BinaryPrecisionRecallCurve
# from supervised.automl import AutoML
import warnings
from loguru import logger
from sklearn.metrics import f1_score, precision_score, recall_score, hamming_loss
from catboost.utils import eval_metric
from catboost import metrics as cat_metrics
from sklearn.utils import class_weight

warnings.filterwarnings("ignore")


/home/alextay96/.local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/alextay96/.local/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda20CUDACachingAllocator9allocatorE
  warn(f"Failed to load image Python extension: {e}")


In [2]:
wr.config.s3_endpoint_url = "http://192.168.1.4:8333"

multilabelDf = wr.s3.read_parquet(
    path=f"s3://multilabel_df_lvl_3/",
    dataset=True,
)
multilabelDf.columns

Index(['vision_bonnet', 'vision_bumper_front', 'vision_engine',
       'vision_grille', 'vision_headlamp_lh', 'vision_headlamp_rh',
       'vision_door_front_lh', 'vision_door_front_rh', 'vision_bumper_rear',
       'vision_misc', 'vision_front_panel', 'vision_non_external',
       'vision_wheel', 'vision_fender_front_lh', 'vision_fender_front_rh',
       'vision_rear_quarter_lh', 'vision_tail_lamp_lh', 'vision_tail_lamp_rh',
       'vision_windscreen_front', 'vision_rear_compartment',
       'vision_rear_panel', 'vision_rear_quarter_rh', 'vision_door_rear_rh',
       'vision_door_rear_lh', 'vision_windscreen_rear', 'bonnet',
       'bumper_front', 'engine', 'grille', 'headlamp_lh', 'headlamp_rh',
       'door_front_lh', 'door_front_rh', 'bumper_rear', 'misc', 'front_panel',
       'non_external', 'wheel', 'fender_front_lh', 'fender_front_rh',
       'rear_quarter_lh', 'tail_lamp_lh', 'tail_lamp_rh', 'windscreen_front',
       'rear_compartment', 'rear_panel', 'rear_quarter_rh', 'door_

In [3]:
wr.config.s3_endpoint_url = "http://192.168.1.4:8333"

multilabelDf = wr.s3.read_parquet(
        path=f"s3://multilabel_df_lvl_3/",
        dataset=True,
    )
allVisionFeatures = [x for x in multilabelDf.columns if "vision_" in x]
allVisionFeatures = [
   'vision_bonnet',
 'vision_bumper_front',
 'vision_engine',
 'vision_grille',
 'vision_headlamp_lh',
 'vision_headlamp_rh',
#  'vision_door_front_lh',
#  'vision_door_front_rh',
 'vision_bumper_rear',
#  'vision_misc',
 'vision_front_panel',
#  'vision_non_external',
#  'vision_wheel',
 'vision_fender_front_lh',
 'vision_fender_front_rh',
 'vision_rear_quarter_lh',
 'vision_tail_lamp_lh',
 'vision_tail_lamp_rh',
 'vision_windscreen_front',
 'vision_rear_compartment',
 'vision_rear_panel',
 'vision_rear_quarter_rh',
#  'vision_door_rear_rh',
#  'vision_door_rear_lh',
 'vision_windscreen_rear']
caseFeatures = [
    "Circumstances_of_Accident",
    "Model",
    # "Vehicle_Still_Driveable",
    # "NCB_Stat",
    # "Assembly_Type",
    # "Claim_Type",
    "Vehicle_Type",
    # # "Sum_Insured",
    # "Repairer",
    # # # "Repairer_Apprv_Count",
    # "Collision_With",
    # "Handling_Insurer",
]
exclude = ['Sum_Insured', 'Repairer', 'Repairer_Apprv_Count', 
"Vehicle_Still_Driveable",
    "NCB_Stat",
    "Assembly_Type",
    "Claim_Type",
    # "Vehicle_Type",
    "Sum_Insured",
    "Repairer",
    "Repairer_Apprv_Count",
    "Collision_With",
    "Handling_Insurer"
     ]
# contFeatures = ["Sum_Insured", "Repairer_Apprv_Count"]
allInputFeature = caseFeatures + allVisionFeatures
# allTargetCol = [
#    'bonnet', 'bumper_front', 'grille', 'fog_lamp_rh', 'headlamp_lh', 'headlamp_rh', 'door_front_lh', 'door_front_rh', 'air_conditioning', 'cooling_fan', 'radiator', 'bumper_rear', 'misc', 'engine', 'airbag', 'front_panel', 'undercarriage', 'wheel', 'fender_front_lh', 'fender_front_rh', 'door_mirror_rh', 'rear_quarter_lh', 'interior', 'tail_lamp_lh', 'fog_lamp_lh', 'tail_lamp_rh', 'windscreen_front', 'rear_compartment', 'rear_panel', 'rear_quarter_rh', 'door_rear_rh', 'door_mirror_lh', 'door_rear_lh', 'windscreen_rear'
# ]

allTargetCol = [
   'bonnet', 
   'bumper_front', 
   'grille', 
   'headlamp_lh',
   'headlamp_rh', 
#    'door_front_lh', 
#    'door_front_rh', 
   'bumper_rear', 
   'engine', 
   'front_panel', 
#    'wheel',
   'fender_front_lh',
   'fender_front_rh', 
   'rear_quarter_lh', 
   'tail_lamp_lh', 
   'tail_lamp_rh',
   'windscreen_front', 
   'rear_compartment',
   'rear_panel', 
   'rear_quarter_rh', 
#    'door_rear_rh', 
#    'door_rear_lh', 
   'windscreen_rear'
]


In [4]:
expId = 134
saloonPredDf = pd.read_csv(
    f"/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/results/{expId}/Saloon - 4 Dr/Saloon - 4 Dr_imgs_pred_output.csv"
)
hatchBackDf = pd.read_csv(f"/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/results/{expId}/Hatchback - 5 Dr/Hatchback - 5 Dr_imgs_pred_output.csv")
suvDf = pd.read_csv(f"/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/results/{expId}/SUV - 5 Dr/SUV - 5 Dr_imgs_pred_output.csv")

imgPredOutput = pd.concat([saloonPredDf, hatchBackDf, suvDf])
# imgPredOutput = saloonPredDf
allPredModel = []
allPreds = []
allGt = []
realTestDataDf = multilabelDf[caseFeatures + allTargetCol + ["CaseID"]].merge(
    imgPredOutput, on="CaseID"
)
realTestDataDf = realTestDataDf.loc[
    :, ~realTestDataDf.columns.str.contains("^Unnamed")
]
trainDf = multilabelDf[
    ~multilabelDf["CaseID"].isin(realTestDataDf["CaseID"].unique().tolist())
]
# trainDf = trainDf.sample(frac=0.5)
assert set(realTestDataDf["CaseID"].tolist()).isdisjoint(trainDf["CaseID"].tolist())
# targetPart = [
#     "fog_lamp_rh",
#     "airbag",
#     "undercarriage",
#     "door_mirror_rh",
#     "interior",
#     "fog_lamp_lh",
#     "door_mirror_lh",
#     "windscreen_rear",
#     "air_conditioning",
#     "cooling_fan",
#     "radiator"
# ]
# targetPart = [
#      'bonnet', 'bumper_front', 'grille', 'headlamp_lh', 'headlamp_rh', 'door_front_lh', 'door_front_rh', 'bumper_rear', 'misc', 'engine', 'front_panel', 'wheel', 'fender_front_lh', 'fender_front_rh', 'rear_quarter_lh', 'interior', 'tail_lamp_lh', 'tail_lamp_rh', 'windscreen_front', 'rear_compartment', 'rear_panel', 'rear_quarter_rh', 'door_rear_rh', 'door_rear_lh'
# ]
# targetCol = list(set(targetPart) & set(allTargetCol))
targetCol = allTargetCol


In [5]:
allVisionFeatures

['vision_bonnet',
 'vision_bumper_front',
 'vision_engine',
 'vision_grille',
 'vision_headlamp_lh',
 'vision_headlamp_rh',
 'vision_bumper_rear',
 'vision_front_panel',
 'vision_fender_front_lh',
 'vision_fender_front_rh',
 'vision_rear_quarter_lh',
 'vision_tail_lamp_lh',
 'vision_tail_lamp_rh',
 'vision_windscreen_front',
 'vision_rear_compartment',
 'vision_rear_panel',
 'vision_rear_quarter_rh',
 'vision_windscreen_rear']

In [6]:

for part in tqdm(targetCol):
    logger.success(f"Start training for part : {part}")
    allCaseIdByPart = []
    allPredByPart = []
    allGtByPart = []
    trainCaseId = trainDf["CaseID"].tolist()
    tempTestDataDf = realTestDataDf
    # tempTestDataDf = realTestDataDf.groupby(part).head(5000)
    testCaseId = tempTestDataDf["CaseID"].tolist()
    # sampledDf = trainDf[allInputFeature + [part]].groupby(part).sample(frac=0.3)
    X_train = trainDf[allInputFeature]
    Y_train = trainDf[part].to_frame()
   
    X_test = tempTestDataDf[allInputFeature]
    Y_test = tempTestDataDf[part].to_frame()

    pos_count = len(Y_test[Y_test[part] == 1]) / len(Y_test)
    neg_count = len(Y_test[Y_test[part] == 0]) / len(Y_test)
    pos_weight = neg_count / pos_count
    train_pool = Pool(
        X_train, Y_train, cat_features=caseFeatures + allVisionFeatures
    )
    test_pool = Pool(X_test, Y_test, cat_features=caseFeatures + allVisionFeatures)
    clsWeight=class_weight.compute_class_weight(class_weight='balanced',classes=np.unique(Y_train.values),y=Y_train.values.squeeze(1))
    logger.success(f"train_class_weight : {clsWeight}")

    testClsWeights = {
        0 : clsWeight[0],
        1 : clsWeight[1]

    }
    print(testClsWeights)
    sampleWeights = class_weight.compute_sample_weight(class_weight=testClsWeights, y=Y_test.values)
    print(f"Shape :{sampleWeights.shape}")
    # prCurve = MulticlassPrecisionRecallCurve(num_classes=2, thresholds=11)
    clf = CatBoostClassifier(
            loss_function="Logloss",
            eval_metric="F1",
            iterations=300,
            early_stopping_rounds=100,
            task_type="GPU",
            # class_weights= clsWeight,
            # auto_class_weights="SqrtBalanced",
            use_best_model=True,
            # scale_pos_weight=20
            # class_names=["not_dmg", "dmg"],
        )
    clf.fit(train_pool, eval_set=test_pool, metric_period=50, plot=False, verbose=1)
    test_predict = clf.predict(test_pool)
    # if (part in partFPThreshold):
    #     logger.success(f"Trade off false positive : {part}")
    #     (fpr, tpr, thresholds) = get_roc_curve(clf, test_pool, plot=False)
    #     boundary = select_threshold(clf, curve=(fpr, tpr, thresholds), FPR=partFPThreshold[part])
    #     clf.set_probability_threshold(boundary)
    #     test_predict = clf.predict(X_test)
    # elif part in partFNThreshold:
    #     logger.success(f"Trade off false negative : {part}")
    #     (fpr, tpr, thresholds) = get_roc_curve(clf, test_pool, plot=False)
    #     boundary = select_threshold(clf, curve=(fpr, tpr, thresholds), FNR=partFNThreshold[part])
    #     clf.set_probability_threshold(boundary)
    #     test_predict = clf.predict(X_test)
    acc = accuracy_score(Y_test.values.astype(np.int64), test_predict)
    confMat = confusion_matrix(Y_test.values.astype(np.int64), test_predict)
    pos_count = len(Y_test[Y_test[part] == 1]) / len(Y_test)
    pos_count_abs = len(Y_test[Y_test[part] == 1])
    # catboostF1 = cat_metrics.F1(Y_test.values.astype(np.int64), test_predict)
    tn = confMat[0][0]
    tp = confMat[1][1]
    fp = confMat[0][1]
    fn = confMat[1][0]
    totalSample = fp + fn + tp + tn
    acc = (tp + tn) / (fp + fn + tp + tn)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 =  (2 * precision * recall) / (precision + recall)
    f1_weighted = f1_score(Y_test.values.astype(np.int64), test_predict, sample_weight=sampleWeights)
    # precision_weighted = precision_score(Y_test.values.astype(np.int64), test_predict, sample_weight=sampleWeights)
    # recall_weighted = recall_score(Y_test.values.astype(np.int64), test_predict, sample_weight=sampleWeights)

    logger.success(f"F1 for part : {part} = {f1}")
    logger.success(f"Weighted F1 scikit for part : {part} = {f1_weighted}")

    # assert (tp / (tp + fn)) + (fn / (tp + fn)) == (tn / (tn + fp)) + (
    #     fp / (tn + fp)
    # )

    allPredModel.append(
        {
            "part": part,
            "tp": tp / (tp + fn),
            "tn": tn / (tn + fp),
            "fp": fp / (tn + fp),
            "fn": fn / (tp + fn),
            "acc": acc,
            "precision": precision,
            "recall": recall,
            "f1": f1,
            "pos_count": pos_count,
            "pos_count_abs": pos_count_abs,

        }
    )
    assert len(testCaseId) == len(test_predict)
    assert len(testCaseId) == len(Y_test)

    allPreds.append({"CaseID": testCaseId, part: test_predict})
    allGt.append({"CaseID": testCaseId, part: Y_test.values.squeeze(1)})


  0%|          | 0/18 [00:00<?, ?it/s]2023-01-13 17:01:31.004 | SUCCESS  | __main__:<module>:2 - Start training for part : bonnet
2023-01-13 17:01:32.066 | SUCCESS  | __main__:<module>:25 - train_class_weight : [0.78110691 1.38934135]


{0: 0.7811069105905991, 1: 1.3893413522803684}
Shape :(4786,)
Learning rate set to 0.068142
0:	learn: 1.0000000	test: 0.7448912	best: 0.7448912 (0)	total: 23.6ms	remaining: 7.05s
50:	learn: 1.0000000	test: 0.7448912	best: 0.7448912 (0)	total: 1.45s	remaining: 7.09s


2023-01-13 17:01:35.864 | SUCCESS  | __main__:<module>:80 - F1 for part : bonnet = 0.7448912326961106
2023-01-13 17:01:35.865 | SUCCESS  | __main__:<module>:81 - Weighted F1 scikit for part : bonnet = 0.7805965037241289
  6%|▌         | 1/18 [00:04<01:22,  4.86s/it]2023-01-13 17:01:35.866 | SUCCESS  | __main__:<module>:2 - Start training for part : bumper_front


100:	learn: 1.0000000	test: 0.7448912	best: 0.7448912 (0)	total: 2.88s	remaining: 5.68s
bestTest = 0.7448912327
bestIteration = 0
Shrink model to first 1 iterations.


2023-01-13 17:01:36.927 | SUCCESS  | __main__:<module>:25 - train_class_weight : [1.39501553 0.77932476]


...

100:	learn: 1.0000000	test: 0.2857143	best: 0.2857143 (0)	total: 2.67s	remaining: 5.27s
bestTest = 0.2857142857
bestIteration = 0
Shrink model to first 1 iterations.


2023-01-13 17:02:43.374 | SUCCESS  | __main__:<module>:25 - train_class_weight : [ 0.52120183 12.2914344 ]


{0: 0.5212018310592096, 1: 12.291434395540394}
Shape :(4786,)
Learning rate set to 0.068142
0:	learn: 1.0000000	test: 0.3338898	best: 0.3338898 (0)	total: 22.5ms	remaining: 6.73s
50:	learn: 1.0000000	test: 0.3338898	best: 0.3338898 (0)	total: 1.36s	remaining: 6.66s


2023-01-13 17:02:46.462 | SUCCESS  | __main__:<module>:80 - F1 for part : windscreen_rear = 0.333889816360601
2023-01-13 17:02:46.463 | SUCCESS  | __main__:<module>:81 - Weighted F1 scikit for part : windscreen_rear = 0.6704328232632413
100%|██████████| 18/18 [01:15<00:00,  4.19s/it]

100:	learn: 1.0000000	test: 0.3338898	best: 0.3338898 (0)	total: 2.71s	remaining: 5.34s
bestTest = 0.3338898164
bestIteration = 0
Shrink model to first 1 iterations.


In [7]:

evalMetrics = pd.json_normalize(allPredModel)
avgPrecision = evalMetrics["precision"].mean()
avgRecall = evalMetrics["recall"].mean()
avgF1 = evalMetrics["f1"].mean()
avgTp = evalMetrics["tp"].mean()
avgFp = evalMetrics["fp"].mean()

avgTn = evalMetrics["tn"].mean()
avgAcc = evalMetrics["acc"].mean()
avgFn = evalMetrics["fn"].mean()
avgPosCount = evalMetrics["pos_count"].mean()

print(f"Avg Precision : {avgPrecision}")
print(f"Avg Recall : {avgRecall}")
print(f"Avg F1 : {avgF1}")
print(f"Avg TP : {avgTp}")
print(f"Avg TN : {avgTn}")
print(f"Avg FN : {avgFn}")
print(f"Avg pos_count : {avgPosCount}")

print(f"avgAcc : {avgAcc}")

Avg Precision : 0.5223410654843054
Avg Recall : 0.7051202133042722
Avg F1 : 0.5814214697419835
Avg TP : 0.7051202133042722
Avg TN : 0.8419953243716433
Avg FN : 0.2948797866957278
Avg pos_count : 0.22829316989367132
avgAcc : 0.8139248734735571


In [8]:
fig = px.bar(evalMetrics, x = 'part', y = 'pos_count_abs', text_auto=True)
fig

# fig.update_layout(
#     yaxis = dict(
#         tickmode = 'array',
#         tickvals = [np.format_float_positional(avgTp, 2), 0.2, 0.5, 0.7],
#     )
# )

In [9]:
fig = px.bar(evalMetrics, x = 'part', y = 'pos_count')

fig.add_hline(y=avgPosCount, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgTp, 2), 0.2, 0.5, 0.7],
    )
)

In [10]:
fig = px.bar(evalMetrics, x = 'part', y = 'tp')

fig.add_hline(y=avgTp, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgTp, 2), 0.2, 0.5, 0.7],
    )
)
fig

In [11]:
fig = px.bar(evalMetrics, x = 'part', y = 'fp')

fig.add_hline(y=avgFp, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgFp, 2), 0.2, 0.5, 0.7],
    )
)

In [12]:
fig = px.bar(evalMetrics, x = 'part', y = 'tn')

fig.add_hline(y=avgTn, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgTn, 2), 0.2, 0.5, 0.7],
    )
)
fig

In [13]:
fig = px.bar(evalMetrics, x = 'part', y = 'fn')

fig.add_hline(y=avgFn, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgFn, 2), 0.2, 0.5, 0.7],
    )
)
fig

In [14]:
fig = px.bar(evalMetrics, x = 'part', y = 'acc')

fig.add_hline(y=avgAcc, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgAcc, 2), 0.2, 0.5, 0.7],
    )
)

In [15]:
fig = px.bar(evalMetrics, x = 'part', y = 'precision')

fig.add_hline(y=avgPrecision, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgPrecision, 2), 0.2, 0.5, 0.7],
    )
)
fig

In [16]:
fig = px.bar(evalMetrics, x = 'part', y = 'recall')

fig.add_hline(y=avgRecall, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgRecall, 2), 0.2, 0.5, 0.7],
    )
)
fig

In [17]:
fig = px.bar(evalMetrics, x = 'part', y = 'f1')

fig.add_hline(y=avgF1, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgF1, 2), 0.2, 0.5, 0.7],
    )
)
fig

In [18]:
multilabelPred = pd.DataFrame(pd.DataFrame(allPreds[0]))
for p in tqdm(allPreds[1:]):
    dfPart = pd.DataFrame(p)
    multilabelPred = multilabelPred.merge(dfPart, on="CaseID")
multilabelPred.sort_values(by="CaseID", inplace=True)
multilabelPred

100%|██████████| 17/17 [00:00<00:00, 112.60it/s]


,CaseID,bonnet,bumper_front,grille,headlamp_lh,headlamp_rh,bumper_rear,engine,front_panel,fender_front_lh,fender_front_rh,rear_quarter_lh,tail_lamp_lh,tail_lamp_rh,windscreen_front,rear_compartment,rear_panel,rear_quarter_rh,windscreen_rear
279,10000064,1,1,1,1,1,0,1,1,1,1,0,0,0,1,0,0,0,0
2879,10000243,1,1,1,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0
1725,10001809,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3120,10001890,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1726,10002135,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2224,13558073,0,1,0,0,1,1,1,1,0,1,0,0,0,0,0,0,0,1
2225,13558341,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0
1060,13558498,0,1,0,1,0,0,1,0,1,1,0,0,0,0,0,0,0,0
3897,13561670,0,1,0,0,0,1,1,0,1,0,0,0,1,0,1,0,0,0


In [19]:
multilabelGt = pd.DataFrame(pd.DataFrame(allGt[0]))
for p in tqdm(allGt[1:]):
    dfPart = pd.DataFrame(p)
    multilabelGt = multilabelGt.merge(dfPart, on="CaseID")
multilabelGt.sort_values(by="CaseID", inplace=True)

100%|██████████| 17/17 [00:00<00:00, 103.20it/s]


In [20]:
multilabelGt = multilabelGt[multilabelPred.columns]
multilabelGt

,CaseID,bonnet,bumper_front,grille,headlamp_lh,headlamp_rh,bumper_rear,engine,front_panel,fender_front_lh,fender_front_rh,rear_quarter_lh,tail_lamp_lh,tail_lamp_rh,windscreen_front,rear_compartment,rear_panel,rear_quarter_rh,windscreen_rear
279,10000064,1,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0
2879,10000243,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1725,10001809,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3120,10001890,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1726,10002135,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2224,13558073,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0
2225,13558341,0,1,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0
1060,13558498,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3897,13561670,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [21]:
multilabelGt[targetCol].values.astype(np.int32)

array([[1, 1, 0, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [1, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [22]:
hammingLoss = hamming_loss(multilabelGt[targetCol].values.astype(np.int32), multilabelPred[targetCol].values)
subsetAcc2 = 1 - hammingLoss

In [23]:
evalData = pd.DataFrame(
    [
        {
            "metric" : "subset_accuracy",
            "value" : subsetAcc2

        },
         {
            "metric" : "avg_f1",
            "value" : avgF1

        },
         {
            "metric" : "avg_tpr",
            "value" : avgTp

        },
        {
            "metric" : "avg_tnr",
            "value" : avgTn

        },
        {
            "metric" : "part_recall",
            "value" : avgRecall

        },
            {
            "metric" : "part_precision",
            "value" : avgPrecision

        },
                {
            "metric" : "part_accuracy",
            "value" : avgAcc

        }
    ]
)


In [24]:
fig = px.bar(evalData, x = 'metric', y = 'value', text_auto=True)
fig

In [25]:
corr = evalMetrics["tp"].corr(evalMetrics["pos_count"])
corr

0.05478312641543593

In [26]:
corr = evalMetrics["f1"].corr(evalMetrics["pos_count"])
corr

0.840276396862835

In [27]:
corr = evalMetrics["acc"].corr(evalMetrics["pos_count"])
corr

-0.7036972160557972

In [28]:
targetCol[-1]

'windscreen_rear'

In [29]:
clf.classes_

array([0, 1])

In [30]:
# explainer = shap.Explainer(clf)
# shap_values = explainer(X_test)
# plt.title(targetCol[-1])
# shap.plots.beeswarm(shap_values)


In [31]:
currCaseStudyDf = pd.read_csv("/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/test_set/test_case_id.csv")
currCaseStudyDf = currCaseStudyDf.sample(n=30)

In [32]:
caseStudyPred = multilabelPred[multilabelPred['CaseID'].isin(currCaseStudyDf["CaseID"].unique().tolist())]
caseStudyGt = multilabelGt[multilabelGt["CaseID"].isin(currCaseStudyDf["CaseID"].unique().tolist())]
caseStudyPred.to_csv("case_study_pred.csv")
caseStudyGt.to_csv("case_study_gt.csv")

In [33]:
caseStudyPred

,CaseID,bonnet,bumper_front,grille,headlamp_lh,headlamp_rh,bumper_rear,engine,front_panel,fender_front_lh,fender_front_rh,rear_quarter_lh,tail_lamp_lh,tail_lamp_rh,windscreen_front,rear_compartment,rear_panel,rear_quarter_rh,windscreen_rear
849,10071959,1,1,1,1,1,0,1,1,1,1,0,0,0,0,0,0,1,0
517,10245671,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0
694,10266332,1,1,1,1,1,0,1,1,1,1,0,0,1,1,0,0,1,0
753,10305063,1,1,1,1,1,0,1,1,1,1,0,0,0,1,0,0,0,0
2639,10314722,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
946,10315950,1,1,1,1,1,0,1,1,1,1,0,0,0,1,0,0,0,0
1001,10353728,1,1,1,1,1,0,1,1,1,1,0,0,0,1,0,0,0,0
2176,10357658,1,1,1,1,1,0,1,1,1,1,0,0,0,1,0,0,0,1
3505,10365133,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
230,10382946,1,1,1,1,1,0,1,1,1,1,0,0,0,1,0,0,0,0
